## 📦 Step 1: Package Installation

Install the fabric-launcher package and its dependencies.

In [ ]:
%pip install fabric-launcher --quiet
%pip install --upgrade azure-core azure-identity --quiet

In [ ]:
print("⚠️ Restarting Python kernel for installed packages to take effect")
notebookutils.session.restartPython()

## ⚙️ Step 2: Configuration

Update these values to customize the deployment for your environment.

In [ ]:
# GitHub Repository Configuration
REPO_OWNER = "myorg"
REPO_NAME = "my-fabric-solution"
BRANCH = "main"
FOLDER_TO_EXTRACT = "workspace"

# GitHub Personal Access Token (optional, for private repositories)
GITHUB_TOKEN = ""

# Deployment Configuration
ENVIRONMENT = "DEV"  # Options: DEV, TEST, PROD
DEBUG = False

# Fabric API Configuration
API_ROOT_URL = "https://api.fabric.microsoft.com"

## 🚀 Step 3: Initialize Launcher

Import the fabric-launcher package and initialize the launcher.

In [ ]:
import notebookutils

from fabric_launcher import FabricLauncher

# Initialize the launcher
launcher = FabricLauncher(
    notebookutils,
    environment=ENVIRONMENT,
    api_root_url=API_ROOT_URL,
    debug=DEBUG,
    allow_non_empty_workspace=False,  # Safety: only deploy to empty workspaces
)

print("✅ Fabric Launcher initialized")
print(f"📍 Workspace ID: {launcher.workspace_id}")
print(f"🏷️ Environment: {launcher.environment}")

## 📥 Step 4: Download and Deploy

Download the solution from GitHub and deploy all artifacts to the current workspace.

This operation will:
- Download the repository from GitHub
- Extract the specified folder
- Deploy Fabric items in stages (data stores first, then compute & analytics)

In [ ]:
print("=" * 60)
print("🚀 Starting Download and Deployment")
print("=" * 60)

launcher.download_and_deploy(
    repo_owner=REPO_OWNER,
    repo_name=REPO_NAME,
    workspace_folder=FOLDER_TO_EXTRACT,
    branch=BRANCH,
    github_token=GITHUB_TOKEN if GITHUB_TOKEN else None,
    staged_deployment=True,  # Deploy data stores first, then compute
    data_folders={"data": "reference-data", "samples": "sample-data"},  # Optional
    lakehouse_name="ReferenceDataLH",  # Optional: specify if using data_folders
    data_file_patterns=["*.json", "*.csv", "*.geojson"],  # Optional
)

print("\n" + "=" * 60)
print("✅ Download and Deployment Completed!")
print("=" * 60)

## 📁 Step 5: Copy Data Folders to Lakehouse (Optional)

Copy data folders from the downloaded repository to a Lakehouse.

**Note:** If you used `data_folders` parameter in Step 4, this step is already done.

In [ ]:
# Configure data folder copying
LAKEHOUSE_NAME = "ReferenceDataLH"
DATA_FOLDER_MAPPINGS = {
    "data": "reference-data",  # Copy 'data' folder to 'reference-data' in Lakehouse
    "samples": "sample-data",  # Copy 'samples' folder to 'sample-data' in Lakehouse
}
FILE_PATTERNS = ["*.json", "*.csv", "*.geojson"]  # Only copy these file types

print("=" * 60)
print("📁 Copying Data Folders to Lakehouse")
print("=" * 60)

try:
    import os

    repository_base_path = ".lakehouse/default/Files/src"

    launcher.copy_data_folders_to_lakehouse(
        lakehouse_name=LAKEHOUSE_NAME,
        repository_base_path=repository_base_path,
        folder_mappings=DATA_FOLDER_MAPPINGS,
        file_patterns=FILE_PATTERNS,
        recursive=True,
    )
    print(f"✅ Data folders copied to {LAKEHOUSE_NAME}/Files/")
except Exception as e:
    print(f"⚠️ Data folder copy skipped or failed: {e}")
    print("This is optional - deployment can continue without reference data.")

## ▶️ Step 6: Execute Post-Deployment Notebook (Optional)

Trigger a post-deployment notebook to initialize data or perform other setup tasks.

In [ ]:
# Configure post-deployment notebook execution
POST_DEPLOYMENT_NOTEBOOK = "Initialize-Reference-Data"
NOTEBOOK_PARAMETERS = {"environment": ENVIRONMENT, "data_folder": TARGET_FOLDER}

print("=" * 60)
print("▶️ Executing Post-Deployment Notebook")
print("=" * 60)

try:
    result = launcher.run_notebook(notebook_name=POST_DEPLOYMENT_NOTEBOOK, parameters=NOTEBOOK_PARAMETERS)

    print("✅ Notebook execution triggered successfully")
    print(f"📓 Notebook: {POST_DEPLOYMENT_NOTEBOOK}")
    print(f"🆔 Job ID: {result['job_id']}")
    print("ℹ️ The notebook is running in the background.")
except Exception as e:
    print(f"⚠️ Notebook execution skipped or failed: {e}")
    print("This is optional - deployment is complete.")

## ✅ Deployment Summary

Your Fabric solution has been deployed successfully!

### What was deployed:
- ✓ Fabric workspace items (Lakehouses, Notebooks, etc.)
- ✓ Reference data files (if configured)
- ✓ Post-deployment tasks (if configured)

### Next Steps:
1. Refresh your browser to see the newly deployed items
2. Review the deployed artifacts in your workspace
3. Check the execution status of any triggered notebooks
4. Verify data and configurations

### Manual Steps (if any):
- Review solution-specific documentation for any manual configuration steps
- Update connection strings or credentials as needed
- Configure scheduled refreshes or triggers

## 🔧 Advanced: Alternative Deployment Approaches

The sections below show alternative ways to use fabric-launcher for more control over the deployment process.

### Alternative 1: Step-by-Step Deployment

This approach gives you more control over each deployment stage.

In [ ]:
# Step 1: Download from GitHub
print("📥 Downloading from GitHub...")
launcher.download_repository(
    repo_owner=REPO_OWNER,
    repo_name=REPO_NAME,
    extract_to=".lakehouse/default/Files/src",
    folder_to_extract=FOLDER_TO_EXTRACT,
    branch=BRANCH,
)

# Step 2: Deploy data stores first
print("\n🗄️ Deploying data stores...")
import os

repository_directory = os.path.join(".lakehouse/default/Files/src", FOLDER_TO_EXTRACT)

from fabric_launcher import FabricDeployer

deployer = FabricDeployer(
    workspace_id=launcher.workspace_id,
    repository_directory=repository_directory,
    notebookutils=notebookutils,
    environment=ENVIRONMENT,
)
deployer.deploy_data_stores()

# Step 3: Deploy compute and analytics
print("\n⚙️ Deploying compute and analytics...")
deployer.deploy_compute_and_analytics()

print("\n✅ Step-by-step deployment completed!")

### Alternative 2: Selective Item Type Deployment

Deploy only specific types of Fabric items.

In [ ]:
# Deploy only specific item types
launcher.deploy_artifacts(
    repository_directory=".lakehouse/default/Files/src/workspace", item_types=["Lakehouse", "Notebook", "Eventstream"]
)

print("✅ Selective deployment completed!")

### Alternative 3: Using Individual Components

For maximum control, use individual components directly.

In [ ]:
from fabric_launcher import FabricDeployer, GitHubDownloader, LakehouseFileManager, NotebookExecutor

# 1. GitHub operations
downloader = GitHubDownloader(REPO_OWNER, REPO_NAME, branch=BRANCH)
downloader.download_and_extract_folder(extract_to="./src", folder_to_extract=FOLDER_TO_EXTRACT)

# 2. Fabric deployment
deployer = FabricDeployer(
    workspace_id=launcher.workspace_id,
    repository_directory="./src/workspace",
    notebookutils=notebookutils,
    environment=ENVIRONMENT,
)
deployer.deploy_all_in_stages()

# 3. File operations
file_mgr = LakehouseFileManager(notebookutils)
file_mgr.upload_file_to_lakehouse(lakehouse_name=LAKEHOUSE_NAME, file_path="./config.json", target_folder="config")

# 4. Notebook execution
executor = NotebookExecutor(notebookutils)
result = executor.run_notebook_synchronous(notebook_path=POST_DEPLOYMENT_NOTEBOOK, parameters=NOTEBOOK_PARAMETERS)

print("✅ Component-based deployment completed!")